# UCI dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel_iris.ipynb)

In [ ]:
%%capture
!pip install iblm --upgrade

### Training

In [3]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [29]:
#df = pd.read_csv('/content/ucidata_train.csv')
df = pd.read_csv('../datasets/ucidata_train.csv')
x_train = df.drop('income', axis=1)
y_train = df['income']

In [30]:
llm_model_name = 'gpt-4'

params = {
    'columns_name': True
    }

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

In [40]:
file_path = '/content/'


for i in range(30):
    model = iblm.fit(x_train, y_train, model_name = 'uci', file_path=file_path)

> Start of model creating.
Tokens Used: 7559
	Prompt Tokens: 7342
	Completion Tokens: 217
Successful Requests: 1
Total Cost (USD): $0.23328


In [41]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        
        # Calculate the probability based on the given data
        age_factor = row['age'] / 100
        education_num_factor = row['education-num'] / 20
        capital_gain_factor = row['capital-gain'] / (row['capital-gain'] + 1)
        capital_loss_factor = row['capital-loss'] / (row['capital-loss'] + 1)
        hours_per_week_factor = row['hours-per-week'] / 100

        # Combine the factors to calculate the probability
        probability = age_factor * education_num_factor * capital_gain_factor * capital_loss_factor * hours_per_week_factor

        # Normalize the probability to be between 0 and 1
        y = probability / (1 + probability)

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)


## Prediction

In [37]:
#df = pd.read_csv('/content/ucidata_test.csv')
df = pd.read_csv('../datasets/ucidata_test.csv')
x_test = df.drop('income', axis=1)
y_test = df['income']

In [38]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [39]:
accuracy = round(accuracy_score(y_test, y_pred),4)
print(f'Accuracy: {accuracy}')

# Precision
precision = round(precision_score(y_test, y_pred),4)
print(f'Precision: {precision}')

# Recall
recall = round(recall_score(y_test, y_pred),4)
print(f'Recall: {recall}')

# F1 score
f1 = round(f1_score(y_test, y_pred),4)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = round(roc_auc_score(y_test, y_proba),4)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.7591
Precision: 0.0
Recall: 0.0
F1 score: 0.0
ROC-AUC: 0.5


## Prediction from external files


In [12]:
from model_code import titanic

y_proba = titanic.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [13]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.45
Precision: 0.3956043956043956
Recall: 1.0
F1 score: 0.5669291338582677
ROC-AUC: 0.9197048611111112


## Interpretation of results

In [14]:
description = iblm.interpret()

Tokens Used: 896
	Prompt Tokens: 341
	Completion Tokens: 555
Successful Requests: 1
Total Cost (USD): $0.04353


In [15]:
print(description)

- First, the function `predict` takes a DataFrame `x` as input and creates a copy of it named `df`. The columns of `df` are then renamed to be integer indices.

- The function then initializes an empty list called `output` to store the predictions.

- For each row in the DataFrame `df`, the function extracts the following features:
  - `pclass`: Passenger class (First, Second, or Third)
  - `sex`: Gender of the passenger (male or female)
  - `age`: Age of the passenger
  - `fare`: Ticket fare paid by the passenger
  - `embarked`: Port of embarkation (C, Q, or S)
  - `alone`: Whether the passenger is traveling alone or not (True or False)

- The prediction logic is then applied to these features, and a variable `y` is initialized to 0.

- The following conditions are checked and the corresponding values are added to `y`:
  - If `pclass` is 'First', add 0.3 to `y`.
  - If `pclass` is 'Second', add 0.15 to `y`.
  - If `sex` is 'female', add 0.35 to `y`.
  - If `age` is less than or equal 